In [ ]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

# 네이버 영화 리뷰 데이터
# https://movie.naver.com/movie/bi/mi/basic.nhn?code=154255 : 여기서 스크래핑



In [ ]:
# 예전 버전임. scaping.ipynb 와 비교하여 어떤 차이가 있는 지 확인해보자.
'''
# 영화 140자평 스크래핑
url_start = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?' \
      'code=154255&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=' \
      'false&isMileageSubscriptionReject=false&page='

# 영화 리뷰가 몇 페이지까지 있는 지 확인하여 limit에 저장
limit = 2
        
reviews = []
for i in range(1, limit):
    url = url_start + str(i)
    result = requests.get(url)
    soup = BeautifulSoup(result.text, 'html.parser')

    score_result = soup.find('div', class_='score_result')
    for li in score_result.find_all('li'):
        review = li.find('div', class_='score_reple').find('p').text
        #print(review)
        reviews.append(review)

# 파일에 저장
f = open("reviews2.txt", "w")
for r in reviews:
    f.write(r)
    f.write('\n')
f.close()
'''

In [ ]:
#df = pd.read_csv('reviews.txt', delimiter="\t", header=None)

In [ ]:
# 또는 기존에 있는 영화 리뷰 데이터 가져옴
# https://github.com/e9t/nsmc/ 
df = pd.read_csv('ratings_train.txt', delimiter="\t", encoding='utf-8')

In [ ]:
df.head(20)

In [ ]:
df.info()

In [ ]:
reviews = [re.sub(r"[^가-힣A-Za-z0-9]", " ", str(content)) for content in df['document']]

In [ ]:
reviews[:20]

In [ ]:
data = reviews[:20000]

In [ ]:
from konlpy.tag import Okt
okt = Okt()
#pos = lambda d: ['/'.join(p) for p in okt.pos(d, stem=True, norm=True)]
pos = lambda d: ['/'.join(p) for p in okt.pos(d, stem=True, norm=True) 
                 if ('Noun' in p or 'Adjective' in p or 'Verb' in p or 'Adverb' in p) 
                 and '보다' not in p and '하다' not in p]
reviews_tokened = [pos(doc) for doc in data]

In [ ]:
reviews_tokened[:2]

In [ ]:
print(reviews_tokened[1])
reviews_tokened[1][0].split('/')

In [ ]:
reviews_tokened[1][0].split('/')

reviews_tokened_filtered = []
for doc in reviews_tokened:
    newdoc = [d for d in doc if d.split('/')[1] == 'Noun' or d.split('/')[1] == 'Adjective' or d.split('/')[1] == 'Adverb']
    reviews_tokened_filtered.append(newdoc)

In [ ]:
reviews_tokened_filtered[0]

In [ ]:
# LDA
from gensim import corpora

dictionary = corpora.Dictionary(reviews_tokened_filtered)
corpus = [dictionary.doc2bow(review) for review in reviews_tokened_filtered]

In [ ]:
import gensim
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=20, id2word = dictionary)

In [ ]:
print(ldamodel.print_topics(num_topics=20, num_words=10))

In [ ]:
ldamodel.show_topic(0,15)

In [ ]:
ldamodel.show_topic(10,15)

In [ ]:
ldamodel.show_topic(4,15)

In [ ]:
for i in range(20):
    print("TOPIC", i)
    for word in ldamodel.show_topic(i,15):
        print("(%s %.4f)" %(word[0], word[1]), end =" ")
    print()
    print("---"*20)

In [ ]:
ldamodel.get_document_topics(corpus[1])

In [ ]:
import pyLDAvis
import pyLDAvis.gensim as gensimvis

prepared_data = gensimvis.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(prepared_data)
